In [1]:
R=QQ['x']
R.inject_variables()

Defining x


In [24]:
def midet_modular(M):
    #Suponemos M matriz cuadrada con coeficientes en QQ[x].
    n=M.nrows()
    L=0
    for i in range(n):
        for j in range(n):
            # L = max(L,M[i,j].degree())
            if M[i,j].degree() > L:
                L = M[i,j].degree()
    #Con esto tenemos guardado en L el máximo de todos los grados de las entradas de M.
    #Una cota del determinante es nL. Tenemos que hacer interpolación en r=nL+1 puntos.
    r=n*L+1
    LAGRANGE = [(i,M(x=i).det()) for i in range(r)]
    return R.lagrange_polynomial(LAGRANGE)

In [35]:
#M=random_matrix(R,20,20)
#M = matrix(R, 5, 5, lambda i,j: i+j); M #Se pueden construir matrices indicando cómo se construye el término i,j
M = matrix(R, 20, 20, lambda i,j: R.random_element(10));


In [36]:
%time _=M.det()

CPU times: user 4.14 s, sys: 5.5 ms, total: 4.14 s
Wall time: 4.13 s


In [38]:
%time _=midet_modular(M)

CPU times: user 1.07 s, sys: 0 ns, total: 1.07 s
Wall time: 1.07 s


In [39]:
#Si son ocho líneas, ¿por qué no está implementado en Sage?
#Varias razones:
# Esto vale en cuerpos, pero falla en Z/(6)
# Para matrices sobre polinomios, no tiene implementados matrices sobre polinomios sobre un cuerpo y haría falta implementarlo.
# Debemos estar seguros de cuándo es más rápido que lo que hace Sage porque hay dos parámetros.
# El determinante sobre Q lo calcula Sage, pero puede que en cuerpos finitos haga cosas distintas.
# Necesitamos controlar también cuántas veces tiene que calcular los determinantes.

In [78]:
# Queremos ver cómo adaptar el algoritmo modular sobre Z:
def midet_modular_ZZ(M):
    n = M.nrows()
    #Podemos utilizar la cota de Hadamard. Queremos evitar las raíces cuadradas porque Sage las calcula simbólicamente y no es rápido.
    L=prod(fila*fila for fila in M.rows()) #La norma al cuadrado.
    #Necesitamos una cota del temaño del determinante en número de bits.
    #Vamos a calcular una aproximación numérica
    #Necesitamos que (p_1,...,p_r-1)^2>4L. Vamos tomando primos hasta que los tenemos.
    #Vamos evaluando la matriz en distintos cuerpos Z/(p) y calculando los determinantes.
    p=2
    T=2
    CHINO_modulos=[2]
    CHINO_residuo = [ZZ(M.change_ring(GF(2)).det())]
    while (T-1)**2<=4*L:
        p = next_prime(p)
        T= T*p
        CHINO_modulos.append(p)
        determinante = ZZ(M.change_ring(GF(p)).det())
        CHINO_residuo.append(determinante)
    detmodT=crt(CHINO_residuo,CHINO_modulos)
    #Ojo lo siguiente es <0 o <=0, revisar el teorema.
    if 2*detmodT<=T:
        return detmodT
    else:
        return detmodT-T

In [94]:
%time M=matrix(ZZ,150,150,lambda i,j: ZZ.random_element(-10000,10000))
midet_modular_ZZ(M)

CPU times: user 14.1 ms, sys: 105 µs, total: 14.2 ms
Wall time: 13.6 ms


-36448554470603209837720371557386227966422871597781232905394844305495625838069293563987609194912354480471082384825005002866912400950040392314778048254038309507455527646872216549474220187661742941755311527103740406083597519566365396884896553851153694735164385467127634767754117988824224213739085595650453275429707175802791375262129193771655948001190620470510746775817473343095048447118043093874961639363539058327571987207266825815444375760540768413680296824290988175421125651309349353940761341100221361797779631806591454103258674606936561553523531343941148111832974288689411291432983570392978245492015711908695037459527122443215106008948546829082734751157261738929946879322909888356798421409699446

In [85]:
M

20 x 20 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [86]:
M2=matrix(ZZ,10,10,lambda i,j: ZZ.random_element(10))

In [87]:
midet_modular_ZZ(M2)

4651750

In [ ]:
# BIG PRIME vs SMALL PRIME.
# ¿Es mejor tomar un primo grande o empezar en 2 e ir subiendo?
# Es mejor aprovecharse del teorema chino


In [1]:
help(crt)

Help on function crt in module sage.arith.misc:

crt(a, b, m=None, n=None)
    Return a solution to a Chinese Remainder Theorem problem.
    
    INPUT:
    
    - ``a``, ``b`` - two residues (elements of some ring for which
      extended gcd is available), or two lists, one of residues and
      one of moduli.
    
    - ``m``, ``n`` - (default: ``None``) two moduli, or ``None``.
    
    OUTPUT:
    
    If ``m``, ``n`` are not ``None``, returns a solution `x` to the
    simultaneous congruences `x\equiv a \bmod m` and `x\equiv b \bmod
    n`, if one exists. By the Chinese Remainder Theorem, a solution to the
    simultaneous congruences exists if and only if
    `a\equiv b\pmod{\gcd(m,n)}`. The solution `x` is only well-defined modulo
    `\text{lcm}(m,n)`.
    
    If ``a`` and ``b`` are lists, returns a simultaneous solution to
    the congruences `x\equiv a_i\pmod{b_i}`, if one exists.
    
    .. SEEALSO::
    
        - :func:`CRT_list`
    
    EXAMPLES:
    
    Using ``crt`